In [9]:
import spacy
from unidecode import unidecode
import spacy.cli
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np

## Processo mais acurado até agora, modelo salvo localmente

In [10]:
# Carrega o modelo de português
nlp = spacy.load("modelos/pt_core_news_sm/pt_core_news_sm-3.8.0")

In [11]:
def remover_acentos(texto):
    return unidecode(texto)

def preprocessar(texto):
    # NÃO remova acentos antes do spaCy, para manter is_stop funcionando corretamente
    doc = nlp(texto.lower())

    lemas = [
        remover_acentos(token.lemma_.lower())  # só remove acento após lematização
        for token in doc
        if not token.is_stop and token.is_alpha
    ]
    return set(lemas)

def jaccard_lemmatizado(frase1, frase2):
    set1 = preprocessar(frase1)
    set2 = preprocessar(frase2)
    intersecao = set1.intersection(set2)
    uniao = set1.union(set2)
    return len(intersecao) / len(uniao) if uniao else 0.0

# 🔎 Exemplo de uso
f1 = "A casa é azul"
f2 = "A casa é verde"

similaridade = jaccard_lemmatizado(f1, f2)
print("Similaridade de Jaccard com lematização:", round(similaridade, 3))

Similaridade de Jaccard com lematização: 0.333


## Testando similaridades

In [12]:
def cosine_bow_similarity(frase1, frase2):
    vetorizar = CountVectorizer().fit_transform([frase1, frase2])
    matriz = vetorizar.toarray()
    similaridade = cosine_similarity(matriz)
    return similaridade[0, 1]

def jaccard_similarity(frase1, frase2):
    set1 = set(frase1.lower().split())
    set2 = set(frase2.lower().split())
    intersecao = set1.intersection(set2)
    uniao = set1.union(set2)
    return len(intersecao) / len(uniao) if uniao else 0.0

# Exemplos
f1 = "A casa é azul"
f2 = "A casa é verde"

print("Cosseno (Bag of Words):", round(cosine_bow_similarity(f1, f2), 3))
print("Jaccard:", round(jaccard_similarity(f1, f2), 3))

Cosseno (Bag of Words): 0.5
Jaccard: 0.6


## Similaridade de Jaccard retirando stopwords e acentos

In [13]:
# Baixar stopwords se necessário
def remover_acentos(texto):
    return unidecode(texto)

def jaccard_sem_stopwords_acentos(frase1, frase2, idioma='portuguese'):
    stop_words = set(remover_acentos(w.lower()) for w in stopwords.words(idioma))
    
    tokens1 = set(remover_acentos(palavra.lower()) for palavra in frase1.split() if remover_acentos(palavra.lower()) not in stop_words)
    tokens2 = set(remover_acentos(palavra.lower()) for palavra in frase2.split() if remover_acentos(palavra.lower()) not in stop_words)
    
    intersecao = tokens1.intersection(tokens2)
    uniao = tokens1.union(tokens2)
    return len(intersecao) / len(uniao) if uniao else 0.0

# Exemplo
f1 = "A casa é azul"
f2 = "A casa é verde"

print("Jaccard (sem acentos e stopwords):", round(jaccard_sem_stopwords_acentos(f1, f2), 3))

Jaccard (sem acentos e stopwords): 0.333


## Similaridade de Jaccard retirando stopwords

In [14]:
def jaccard_similarity_sem_stopwords(frase1, frase2, idioma='portuguese'):
    stop_words = set(stopwords.words(idioma))
    
    set1 = set(palavra.lower() for palavra in frase1.split() if palavra.lower() not in stop_words)
    set2 = set(palavra.lower() for palavra in frase2.split() if palavra.lower() not in stop_words)
    
    intersecao = set1.intersection(set2)
    uniao = set1.union(set2)
    print(uniao)
    return len(intersecao) / len(uniao)

# Exemplo
f1 = "A casa é azul"
f2 = "A casa é verde"

print("Jaccard com remoção de stopwords:", round(jaccard_similarity_sem_stopwords(f1, f2), 3))

{'azul', 'verde', 'casa'}
Jaccard com remoção de stopwords: 0.333


## Embeddings

In [15]:
# Carregar o modelo de embedding
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Duas palavras similares
palavra1 = "carro"
palavra2 = "gato"

# Gerar embeddings
embedding1 = model.encode(palavra1)
#embedding2 = model.encode(palavra2)

# Calcular similaridade do cosseno
#similaridade = cosine_similarity(
#    [embedding1],
#    [embedding2]
#)[0][0]

#print(f"Similaridade entre '{palavra1}' e '{palavra2}': {similaridade:.4f}")
print(embedding1[:10])

c:\Users\alber\anaconda3\envs\env_gen_ai\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[ 0.21573691  0.18270384 -0.01125517  0.03685256 -0.01532448 -0.00880565
  0.3670183   0.17374927 -0.0828202  -0.2361189 ]


In [ ]:
# Carregando um modelo leve e eficaz
model = SentenceTransformer("all-MiniLM-L6-v2")

In [18]:
# Palavra de exemplo
palavra = "inteligência"

# Gerando o embedding
embedding1 = model.encode(palavra)

# Mostrando as primeiras 10 dimensões
print(embedding1[:10])
print(embedding1.shape)


[ 0.02514462  0.02419465 -0.0337151   0.00544109 -0.05224889 -0.05682078
  0.09118187  0.04109337 -0.03356854  0.01619318]
(384,)
